# Test de conectividad con Twilio-Sandbox

--------------------

## Equipo 36

| Nombre | Matrícula |
| ------ | --------- |
| André Martins Cordebello | A00572928 |
| Enrique Eduardo Solís Da Costa | A00572678 |
| Delbert Francisco Custodio Vargas | A01795613 |


Para llevar a cabo este test de conectividad, hicimos uso de Twilio-Sandbox y la siguiente documentación:

- https://ngrok.com/partners/twilio
- https://www.twilio.com/docs/usage/tutorials/how-use-ngrok-windows-and-visual-studio-test-webhooks
- https://www.twilio.com/docs/messaging/guides/webhook-request
- https://www.twilio.com/docs/whatsapp/message-features#location-messages-with-whatsapp


En los links anteriores es posible encontrar la forma de:

1. Obtener los mensajes de texto por medio de un tuner creado con Ngrok
2. Conectar este tunel a nuestro Kernel de Python por medio de Flask.
3. Recibir y enviar mensajes por medio de WhatsApp.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
import gc
from twilio.twiml.messaging_response import MessagingResponse
from flask import Flask, request, Response
from twilio.rest import Client
from dotenv import load_dotenv
import requests
import json
import os
import re


load_dotenv(dotenv_path="chatbot.env")

account_sid = os.getenv('account_sid_2')
auth_token  = os.getenv('auth_token_2')
FROM_NUMBER  = "" # FROM number de WahtsApp Sandbox de Twilio
client = Client(account_sid, auth_token )

# Flujo principal para reporte de predios infestados con el gorgojo del agave

In [ ]:
# Global session storage (temporary — resets if the app restarts)
user_session = {}

Definimos una función para enviar mensajes sin formato a WhatsApp

In [ ]:
def send_whatsapp_message(From, To, message):

    try:
        
        msg = client.messages.create(
            from_=From,
            to=To,
            body=message
        )
        print(f"Message sent: {msg.sid}")
        return msg.sid
    
    except Exception as e:
        
        print(f"Error sending message: {e}")
        return None

Probamos la función de envío.

In [ ]:
def send_menu(From, To):
    
    menu_message = (
        "👋 *Bienvenido, será un gusto atenderte.*\n\n"
        "¿Qué te interesa llevar a cabo?\n\n"
        "🅰️ *Hacer un reporte de un predio infectado*\n"
        "🅱️ *Preguntar por información*"
        "\n\n"
        "Actualmente, solo estas 2 opciones tenemos disponibles."
    )

    try:
        msg = client.messages.create(
            from_=From,
            to=To,
            body=menu_message
        )
        print(f"✅ Se envio correctamente el menu: {msg.sid}")
        return msg.sid
    except Exception as e:
        print(f"❌ Error enviando menu: {e}")
        return None

In [ ]:
def report_message_flow( sender, state, req ):

	# Cargamos el mensaje
	message = request.form.get("Body", "").strip().lower() # Texto enviado por el usuario
	label   = request.form.get("Label")  				   # Label de la ubicacion enviada
	lat = request.form.get("Latitude") 					   # Si envian ubicacion, cargamos la latitud
	lon = request.form.get('Longitude') 				   # Si envian ubicacion, cargamos la longitud
	address = request.form.get('Address') 				   # Tomamos la dirección generada por WhatsApp
	photo_url = req.form.get('MediaUrl0')
	num_media = int(req.form.get('NumMedia', 0))
 
	response_text = ""
	next_state = state
 
	if state == "saludo":
		if (message == "a") or (message == "🅰️"):
			response_text = ("Gracias por tu proactividad. Te guiaré para reportar el predio. Primero, envía la **ubicación** del lote afectado.")
			next_state  = "ask-location"
   
	elif state == "ask-location":
		if lat and lon:
			user_session[sender]["data"]["lat"]  = lat
			user_session[sender]["data"]["lon"] = lon
			user_session[sender]["data"]["address"]   = address
			response_text = (
				"📍 Ubicación recibida *correctamente*.\n"
				"Ahora, por favor envía una **foto** del lote o planta afectada."
			)
			next_state = "ask_photo"
		else:
			response_text = ("Parece que la información que enviaste no es correcta. Por favor, usa la función de ubicación de WhatsApp.")

	elif state == "ask_photo":
		if num_media > 0:
			photo_url = req.form.get('MediaUrl0')
			user_session[sender]["data"]["photo_url"] = photo_url
			response_text = (
				"📸 Foto recibida correctamente.\n"
				"Por último, clasifica el **nivel de riesgo** que observas: Alto, Medio o Bajo."
			)
			next_state = "ask_risk"
		else:
			response_text = (
				"⚠️ No se detectó una foto. Por favor envía una imagen del lote afectado."
			)

	elif state == "ask_risk":
    
		if message in ["alta", "media", "baja", "alto", "medio", "bajo"]:
			user_session[sender]["data"]["user_risk_assesment"] = message.capitalize()
			data = user_session[sender]["data"]
			response_text = (
				f"✅ Gracias por tu reporte.\n"
				f"📍 Ubicación: ({data['lat']}, {data['lon']})\n"
				f"📸 Foto: Confirmada\n"
				f"🚨 Riesgo: {data['user_risk_assesment']}\n\n"
				"¿Esta información es correcta? Responde con *Sí* o *No*."
			)
			next_state = "confirmation"

		else:
			response_text = (
				"Por favor indica el nivel de riesgo como: Alta, Media o Baja."
			)

	elif state == "confirmation":
		msg = message.strip().lower()

		if re.search(r'^\s*s[ií]\s*$', msg):
			response_text = (
				"🌾 Tu reporte ya fue registrado. ¡Gracias por tu colaboración!"
			)
			print(user_session[sender])
		elif re.search(r'^\s*no\s*$', msg):
			response_text = "❌ Entendido. Tu reporte no se registrará."
		else:
			response_text = "Por el momento, solo podemos aceptar respuestas como 'Sí' o 'No'."
	else:
		response_text = ("Por el momento, solo podemos aceptar respuestas como 'Sí', 'No', etc. ")

	return response_text, next_state

In [ ]:
# reiniciamos user_session para hacer pruebas

user_session = {}

app = Flask(__name__)

@app.route("/reply_whatsapp", methods=['POST'])

def reply_whatsapp():

	sender  = request.form.get("From") # Guardamos quien nos escribio
	message = request.form.get("Body", "").strip().lower() # Tomamos el texto del mensaje para mostrarlo en la consola
	resp = MessagingResponse()
 
	print(f"Mensaje recibido: --> {message}")
 
	if sender not in user_session:
		user_session[sender] = {"state": "saludo",
								"data" : {
									"lat" : None, 
        							"lon" : None,
									"address" : None,
									"photo_url" : None,
									"user_risk_assesment": None,
									"price" : None
								}
              					}

		send_menu(FROM_NUMBER, sender)
		resp.message("Quedo a la espera de tu respuesta.")
		return Response(str(resp), mimetype="text/xml")

	
	response_text, next_state = report_message_flow(sender, user_session[sender]['state'], request)
	user_session[sender]["state"] = next_state
	resp.message(response_text)
	
	return Response(str(resp), mimetype='text/xml')


if __name__ == "__main__":
	app.run(port=3000)

# Flujo para llevar a cabo preguntas y responderlas con el LLM

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
import gc
from twilio.twiml.messaging_response import MessagingResponse
from flask import Flask, request, Response
from twilio.rest import Client
from dotenv import load_dotenv
import requests
import json
import os
import re


load_dotenv(dotenv_path="chatbot.env")

account_sid = os.getenv('account_sid_2')
auth_token  = os.getenv('auth_token_2')
FROM_NUMBER  = "" # El FROM Number de WhatsApp va aqui
client = Client(account_sid, auth_token )

In [ ]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from peft import PeftModel

# Modelo local
model_path = "D:/LLM Models/microsoft-phi-3-mini-4k-instruct"

# LoRA adapters
lora_path = "D:/LLM Models/agave_V001/agave_baseline_phi3_V01"   

# 8 bits de cuantizacion
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
)

# Usamos el tokenizer de Phi-3
tokenizer = AutoTokenizer.from_pretrained(model_path)
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Implementamos los pesos LoRA a Phi-3
model = PeftModel.from_pretrained(base_model, lora_path)

# Colocamos el  modelo en evaluacion para que no cambie los pesos por cada prompt que
# se le envíe
model.eval()

# Definimos el pipeline de nuevo
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    return_full_text=False,
)

In [ ]:
import re

def gen_prompt(tokenizer, sentence):
    converted_sample = [{"role": "user", "content": sentence}]
    prompt = tokenizer.apply_chat_template(
        converted_sample, tokenize=False, add_generation_prompt=True
    )
    return prompt


def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=False):
  
  tokenized_input = tokenizer(
  prompt, add_special_tokens=False, return_tensors="pt").to(model.device)
  model.eval()
  gen_output = model.generate(**tokenized_input,
  eos_token_id=tokenizer.eos_token_id,
  max_new_tokens=max_new_tokens)
  output = tokenizer.batch_decode(gen_output, skip_special_tokens=skip_special_tokens)

  return output[0]

In [ ]:
def answer_with_llm( msg):
    
    prompt = gen_prompt(tokenizer, msg)
    print(prompt)

    answer = generate(model, tokenizer, prompt)
    print(answer)
    
    return answer

In [ ]:
# reiniciamos user_session para hacer pruebas

user_session = {}

app = Flask(__name__)

@app.route("/reply_whatsapp", methods=['POST'])

def reply_whatsapp():

	sender  = request.form.get("From") # Guardamos quien nos escribio
	message = request.form.get("Body", "").strip().lower() # Tomamos el texto del mensaje para mostrarlo en la consola
	resp = MessagingResponse()
 
	print(f"Mensaje recibido: --> {message}")

	
	response_text = answer_with_llm(message)
	resp.message(response_text)
	
	return Response(str(resp), mimetype='text/xml')


if __name__ == "__main__":
	app.run(port=3000)